In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install -q keras

In [ ]:
!pip install -q pydrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
project_path='/content/drive/My Drive/Documentation Smelling/'

In [ ]:
!pip install simpletransformers

In [ ]:
import re
import string
import numpy as np
import pandas as pd


def clean_text(text):
    
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\r", " ", text)


    return text

In [ ]:
import pandas as pd

df = pd.read_excel(project_path + 'Datasets/dataset.xlsx')

df.head()

In [ ]:
#from DataPrep.Clean_Texts import clean_text
import pickle
#dataset=dataset.head(10)

text=df['Documentation Text']
text=text.map(lambda x: clean_text(x))


In [ ]:
label=df.iloc[:,1:6].values

In [ ]:
model_type = "bert"
model_name = "bert-base-cased"

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": project_path + "Models/Transformers/output/"+model_type,
    "best_model_dir": project_path + "Models/Transformers/output/"+model_type+"/best_model",
    "use_early_stopping": False,
    "early_stopping_delta": 0.0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize" : True,
    "early_stopping_patience" : 2,
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 512,
    "wandb_project": "Doc Smelling Bert_1",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "evaluate_during_training_verbose" : True
}

In [ ]:


!pip install -q iterative-stratification

from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel

import glob

import os
#from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import gc
import keras.backend as K
import numpy as np

X_train_all = np.array(text)
y_train_all = np.array(label)

print(X_train_all.shape)
print(y_train_all.shape)

num_cross_validation = 5


mskf = MultilabelStratifiedKFold(n_splits = num_cross_validation, shuffle=True, random_state=42)
pred_list=[]
cvscores = []
y_true_all_fold = []
pred_binary_all_fold = []

Fold = 1

for train, val in mskf.split(X_train_all, y_train_all):
    gc.collect()
    K.clear_session()
    print('Fold: ', Fold)

    X_train = X_train_all[train]
    X_val = X_train_all[val]
    y_train = y_train_all[train]
    y_val = y_train_all[val]

    with open(project_path + 'Datasets/Pickle_5_Fold/dataset_' +str(Fold) + '.pickle','wb') as f:
        pickle.dump((X_train, X_val, y_train, y_val),f)

    Fold = Fold + 1



In [ ]:
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel

import pickle
import glob

import os
#from sklearn.model_selection import KFold
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import gc
import keras.backend as K
import numpy as np
import pandas as pd


num_cross_validation = 5


#mskf = MultilabelStratifiedKFold(n_splits = num_cross_validation, shuffle=True, random_state=42)
pred_list=[]
cvscores = []
y_true_all_fold = []
pred_binary_all_fold = []

Fold = 1

#for i in range(num_cross_validation):

gc.collect()
K.clear_session()
print('Fold: ', Fold)

with open(project_path + 'Datasets/Pickle_5_Fold/dataset_' +str(Fold) + '.pickle','rb') as f:
    X_train, X_val, y_train, y_val = pickle.load(f)



ll=[]
for i in range(len(X_train)):
  ll.append([X_train[i],y_train[i]])

train_val_df = pd.DataFrame(ll)
train_val_df.columns = ["text", "labels"]

ll=[]
for i in range(len(X_val)):
  ll.append([X_val[i],y_val[i]])

test_df = pd.DataFrame(ll)
test_df.columns = ["text", "labels"]



train_df, val_df = train_test_split(train_val_df, test_size=0.1)



train_args["max_seq_length"] = 300
train_args["train_batch_size"] = 32
train_args["gradient_accumulation_steps"] = 2
train_args["evaluate_during_training"] = False
train_args["evaluate_during_training_steps"] = 25
train_args["use_early_stopping"] = False

# Create a ClassificationModel
model = MultiLabelClassificationModel(model_type, model_name, num_labels=5, args=train_args)

# Train the model
model.train_model(train_df, eval_df=val_df)

pred, raw_outputs = model.predict(test_df['text'])



pred_binary=np.array(pred)
for i in range(len(pred_binary)):
  for j in range(len(pred_binary[i])):
    pred_binary[i][j]=int(1*(pred_binary[i][j]>0.5))

  y_true_all_fold.append(y_val[i])
  pred_binary_all_fold.append(pred_binary[i])

with open(project_path + 'Predictions/CV_Bert_1/CV_5_Fold_Predictions/pickle_Pred_doc_Bert_CV_Fold_2_' + str(Fold) +'.pickle','wb') as f:
    pickle.dump((y_true_all_fold,pred_binary_all_fold),f)
    
#Fold = Fold + 1


In [ ]:
from itertools import chain
import pickle

with open(project_path + 'Predictions/CV_Bert_1/CV_5_Fold_Predictions/pickle_Pred_doc_Bert_CV_Fold_2_' + str(1) +'.pickle','rb') as f:
  y_test_all, pred_binary_all = pickle.load(f)

print(len(y_test_all),len(pred_binary_all))

for i in range(2,6):
  with open(project_path + 'Predictions/CV_Bert_1/CV_5_Fold_Predictions/pickle_Pred_doc_Bert_CV_Fold_2_' + str(i) +'.pickle','rb') as f:
    y_test, pred_binary = pickle.load(f)
  
  y_test_all=list(chain(y_test_all,y_test))
  pred_binary_all=list(chain(pred_binary_all,pred_binary))

print(len(y_test_all),len(pred_binary_all))

In [ ]:
from sklearn.metrics import classification_report,precision_recall_fscore_support
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import accuracy_score,jaccard_similarity_score, hamming_loss

report=classification_report(y_test,pred_binary)
#report=precision_recall_fscore_support(y_test,pred_binary,average='micro')
print('Classification Report: '+str(report))


precision=precision_score(y_test,pred_binary,average='weighted')
print('Weighted Precision: '+str(precision))

recall=recall_score(y_test,pred_binary,average='weighted')
print('Weighted Recall: '+str(recall))

f1=f1_score(y_test,pred_binary,average='weighted')
print('Weighted F1 Score: '+str(f1))



acc_hard=accuracy_score(y_test,pred_binary)
print('Hard Accuracy: '+str(acc_hard))

In [ ]:
y_true_np = np.array(y_test)
y_pred_np = np.array(pred_binary)

num_classes = 5
for i in range(num_classes):
  print('Class: ' + str(i))
  new_y_true = y_true_np[:,i]
  new_y_pred = y_pred_np[:,i]

  print('Accuracy for class ' + str(i) + ': ' + str(accuracy_score(new_y_true,new_y_pred)))
  print('Classification Report for class' + str(i) + ': ' + str(classification_report(new_y_true,new_y_pred)))